In [4]:
%pip install --quiet --user --upgrade langchain langchain-community langchain-chroma pypdf
%pip install -qU langchain-together

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:
import getpass
import os

In [2]:
os.environ['USER_AGENT'] = 'myagent'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [8]:
if "TOGETHER_API_KEY" not in os.environ:
    os.environ["TOGETHER_API_KEY"] = getpass.getpass("Enter your Together API key: ")

Enter your Together API key:  ········


# Create LLM

using LLM from together AI API

In [73]:
from langchain_together import ChatTogether

llm = ChatTogether(
    model="meta-llama/Llama-3-70b-chat-hf",
    # model="meta-llama/Llama-2-7b-chat-hf",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Prompt

In [68]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks related to "
    "Pittsburgh and CMU, including history, culture, trivia, and upcoming events.. "
    "Use the following pieces of retrieved-context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Keep the answer concise and use no more than 5 words."
    "Make sure each answer is only one line."
    "Example format is"
    "Q: Who is Pittsburgh named after?"
    "A: William Pitt"
    "Q: What famous machine learning venue had its first conference in Pittsburgh in 1980?"
    "A: ICML"
    "Q: What musical artist is performing at PPG Arena on October 13?"
    "A: Billie Eilish"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)

# Create vector db

In [55]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
# pip install chromadb langchain langchain-huggingface langchain-chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb
from chromadb.utils.embedding_functions import create_langchain_embedding
from langchain_huggingface import HuggingFaceEmbeddings

langchain_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

ef = create_langchain_embedding(langchain_embeddings)

directory_path = "./data/dataset"

vectorstore = Chroma(embedding_function=ef, persist_directory="./pittsburgh_HF_db")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

C:\Users\22720\anaconda3\envs\rag\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [64]:
# pdf -----------

loader = DirectoryLoader(
    directory_path,
    glob="**/*.pdf",  # This pattern will match all .pdf files in the directory and its subdirectories
    loader_cls=PyPDFLoader,
    show_progress=True
)

docs = loader.load()

splits = text_splitter.split_documents(docs)

vectorstore.add_documents(splits)

  5%|███▉                                                                             | 14/286 [00:23<06:16,  1.38s/it]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
  6%|█████                                                                            | 18/286 [00:24<02:49,  1.58it/s]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wron

['07e934d1-ebb0-4836-b4db-b28e408a00da',
 '173e7b51-a823-4235-80cf-dbe5d963e874',
 '064f103e-a18d-438c-85fc-2362d8fcae74',
 '5a8c4d97-a8d9-461a-a57f-596981fae4b5',
 '5e281b4c-fd7a-439a-b414-f3126fb22656',
 'be750260-a253-4ea7-831b-0c86d35c78d6',
 '518b317a-34df-42ea-b754-4322fc012aa4',
 'e00bb999-f506-4c4a-a02e-cde0d9f70110',
 'a569ec8c-f77e-47e2-8cc2-7b901276f55b',
 'c6c5bc2f-0bc1-4c4c-988e-d1c660f301f8',
 '9fe9b3ee-baf0-48a4-bc9e-39d0482590be',
 '116161ab-286d-416e-a6b1-0c5606cf9403',
 '848a25e6-e59b-4c93-a06d-efbc16485302',
 '6659289c-0dc3-4aee-bf3e-54fc25c0a5f7',
 '04487dd7-edb0-4f21-9863-57e4ceb9c829',
 '89e42539-606b-43f9-ac9b-b8d7eeaa1c09',
 '502da142-da33-4915-b504-36ee56a87794',
 '5d81f6fa-ee64-48ad-9abe-340707e45ad9',
 'a69da003-8bb5-4711-8b5d-beb8c4694cb8',
 'eb8b16f8-75ef-4e29-a27a-1f65bd45dead',
 '9f6dbd31-0cf7-421b-a0e3-c18b44d297ed',
 'a2361a99-2b23-4068-b0f2-0f8905f72403',
 '43b4b108-78a7-468d-a675-e5294b0b379a',
 'a0b3eb41-5f7b-453f-a6a2-53ac730026fb',
 '99029936-6f05-

In [69]:
from langchain_community.document_loaders import CSVLoader
# ---- csv
loader = DirectoryLoader(
    directory_path,
    glob="**/*.csv",
    loader_cls=lambda file_path: CSVLoader(file_path, encoding='utf-8'),
    show_progress=True
)
docs = loader.load()

splits = text_splitter.split_documents(docs)

vectorstore.add_documents(splits)



  0%|                                                                                          | 0/817 [00:00<?, ?it/s]

  1%|▋                                                                                 | 7/817 [00:00<00:11, 69.31it/s]

  2%|█▍                                                                               | 15/817 [00:00<00:10, 73.42it/s]

  3%|██▍                                                                              | 24/817 [00:00<00:10, 75.13it/s]

  4%|███▎                                                                             | 33/817 [00:00<00:10, 78.06it/s]

  5%|████▏                                                                            | 42/817 [00:00<00:09, 80.21it/s]

  6%|█████                                                                            | 51/817 [00:00<00:09, 83.09it/s]

  7%|█████▉                                                                           | 60/817 [00:00<00:09, 82.41it/s]

  8%|██████▊                  

['109b46cf-beb1-4ff4-b6a6-9c57f8f1c494',
 '97069675-54ba-4221-91c2-bb0f414aa95c',
 '678383cb-620e-42a1-93db-d413d84d42bd',
 'ae87fbd9-66da-4738-82ef-a9e51b5977c3',
 'a296f483-ec65-44db-b183-f8878fd56dc0',
 'bfcc4d68-a889-4777-8e02-d82296b9d140',
 '06e87711-db13-4a42-9f8e-8cda659499b4',
 '4ce41ed1-8a01-4c48-99bd-be03183c5c55',
 'ddfd0f96-1910-43ab-a59e-55e45bc2daff',
 'ca97d2f5-7355-45f3-80b8-95ed2c8b09a5',
 'd06d3e53-f471-4b3f-85c2-d6f8e2e217c4',
 'c505edda-8543-4c86-95d3-5388bf84806d',
 'a6298b18-1044-4311-88c5-6feb54b3263f',
 '2067ea26-a095-46a8-8596-050377cd1eef',
 '2b917c63-5880-4e4d-ba6a-19758ed6f43f',
 '8e7d562d-be33-4f96-b631-a5b457f61d88',
 'd8014c92-277e-428f-922e-8d97035ffde7',
 '6595d3f7-289c-4733-9987-7ade18a682a7',
 '1f0c9114-19de-41a5-ad2d-1ff71c656362',
 '383e3ba1-e41a-4963-9542-ad14294ff100',
 '3e7f2b58-0a21-4262-98cc-10bd3497233d',
 'fcf939c9-5417-49fc-91c5-09c60d5e1c92',
 'c7f4f98c-619b-4c3f-bfc5-67534d883942',
 'a765c8a8-c595-4559-9d20-52618427e90e',
 '98f68a17-9aa3-

In [124]:
# text --------
vectorstore = Chroma(embedding_function=ef, persist_directory="./pittsburgh_HF_db")

loader = DirectoryLoader(directory_path + "/Music&museums&festivals&sports/sports/", glob='**/*.txt', show_progress=True)
docs = loader.load()

splits = text_splitter.split_documents(docs)

vectorstore.add_documents(splits)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.11it/s]


['dd070893-69cf-4f52-824a-6461d2cfa7c1',
 'f31fda19-92b7-4118-8ece-b94fbf17d266',
 '71c5414e-12f5-4dba-ab30-c2acb8162b28',
 '73fadba8-6a9f-46b9-9713-636c6cec66ec',
 '2e2e5901-0f09-421b-af32-420e98aa1b83',
 'be012328-28c8-4127-ae0f-c299560068f8',
 'c019f44e-15cc-4dee-95ec-aaefce8ecf2f',
 'cf187b5a-3f36-4985-963d-ffdad69b48ca',
 '30a81420-d6cd-4b01-884d-adae79f8aa93',
 'b9f27909-4763-4897-8c15-702453dee396',
 'd68d2d1e-1ffc-4683-ba76-4389c65f08e6',
 '8424b221-ff71-41ec-975f-95e7b26e6464',
 'af3cadb0-457d-4d21-b2fd-4ec94da9b2ad',
 'f82e0cd9-fc7e-45a9-9ea7-9888bd8b0ac6',
 '96288081-9019-49f4-86a5-f5a895831abe',
 'a38ae9af-c61f-476b-973e-7b80c768e897',
 '47ca4fab-506a-4683-8292-c76d568c38cd',
 '57cc309d-7c0a-4947-a099-e96fa26f4d07',
 '286a462e-9f99-45a9-a4bf-84b95992c3f5',
 '77d273c7-344a-4530-96c3-31eb3f0e20a7',
 'efd2c689-fba2-477a-9714-2942a695c735',
 'a48c9102-5a7c-43ca-9a93-2198ed342e2b',
 '7c65f0ba-f817-48dc-8709-d5ed90357c20',
 'e4d5d0a9-99ec-4573-a166-77050d9ca980',
 'c8d0ac94-9c89-

In [ ]:
docs = loader.load()
    
splits = text_splitter.split_documents(docs)

vectorstore.add_documents(splits)


 14%|██████████▊                                                                    | 870/6347 [02:54<20:33,  4.44it/s]No features in text.

 22%|█████████████████                                                             | 1391/6347 [05:52<18:24,  4.49it/s]No features in text.

 22%|█████████████████▎                                                            | 1407/6347 [05:53<08:37,  9.54it/s]No features in text.

 22%|████████████████▉                                                           | 1411/6347 [06:10<3:45:14,  2.74s/it]No features in text.

 22%|████████████████▉                                                           | 1413/6347 [06:10<2:37:45,  1.92s/it]No features in text.

 22%|████████████████▉                                                           | 1415/6347 [06:11<1:51:46,  1.36s/it]No features in text.

 22%|████████████████▉                                                           | 1417/6347 [06:11<1:20:16,  1.02it/s]No features in text.

 22%|███████

# Create traing dataset vector db

In [127]:
# read all question answer pairs into tuple list
import random
def read_qa_files(question_file, answer_file):
    qa_pairs = []
    
    with open(question_file, 'r', encoding='utf-8') as qfile, open(answer_file, 'r', encoding='utf-8') as afile:
        questions = qfile.readlines()
        answers = afile.readlines()
        
        # Ensure both files have the same number of lines
        if len(questions) != len(answers):
            raise ValueError("The question and answer files have different numbers of lines.")
        
        # Create pairs
        for q, a in zip(questions, answers):
            # Strip whitespace and newline characters
            qa_pairs.append((q.strip(), a.strip()))
    
    return qa_pairs

# Usage
directory = "./data/questions-answer-pair/"
areas = ["cmu", "event", "Music_and_Culture", "Pittsburgh", "sports"]
qa_pairs = []
for area in areas:
    qa_pairs.append(read_qa_files(directory + area + "_questions.txt", directory + area + "_answers.txt"))

random.seed(1024)
test_ratio = 0.2
train_data = []
test_data = []
for qa_pair in qa_pairs:
    # Shuffle the data
    random.shuffle(qa_pair)
    
    # Determine the split index
    split_index = int(len(qa_pair) * (1 - test_ratio))
    
    # Split the data
    train_data.append(qa_pair[:split_index])
    test_data.append(qa_pair[split_index:])
print(test_data)

[[('When was the tradition of buggies race started?', '1920'), ('When did Andrew Carnegie donate $1 million for the creation of a technical institute for the city of Pittsburgh?', '1900'), ('when was the first smile :-) in an email created?', '1982'), ("What's the usual max speed of buggies?", '35 mph'), ('Which professor in CMU developed a computer algorithm that accelerates the matching of living donors?', 'Tuomas Sandholm'), ('Who in CMU wrote A History of the Federal Reserve?', 'Allan Meltzer'), ('Who started the first Girl Scout badge for negotiation?', 'Linda Babcock'), ('How many Nobel Laureate has Carnegie Mellon has been home to?', '21'), ('Who in CMU co-founded the influential Shadow Open Market Committee?', 'Allan Meltzer'), ('Which building in Pittsburgh is one of the world’s largest concentrations of certified "green" buildings?', 'the Phipps Conservatory'), ('When was CMU founded?', '1900'), ('Who in CMU developd tutoring systems that are helping 500,000 students in 2,600

In [85]:
# save the training tuple list into the vector_qa_db
from langchain_core.documents import Document

documents = []
for qa_pair in train_data:
    documents.extend([Document(page_content=f"Q: {q}\nA: {a}") for (q, a) in qa_pair])

vectorstore_qa = Chroma(embedding_function=ef, persist_directory="./pittsburgh_QA_1_db")

splits = text_splitter.split_documents(documents)
vectorstore_qa.add_documents(splits)


['8605a010-b93f-43b9-baf8-e7df03112a63',
 '53e7cf28-d13e-4ee5-9b8b-49a724122e2a',
 '7bd42440-2963-4ef0-8ee2-edf60b740bab',
 '00f902a6-1b89-4b9e-8e73-d24c80dc1f68',
 'f2db2390-20a2-4683-9123-d173490a9050',
 '7edc6b6f-755f-413c-9417-f214ade64e38',
 '13043177-b3ce-456e-adb3-42dad2805f05',
 'b9e962e0-df29-405e-ae37-ff56ea67f7b8',
 'd4a0b5e2-cfe8-4e44-b5e7-a44f647c744f',
 '52b8a2e9-8596-4b31-bf7d-2f9f20b02f98',
 'd44d6ac1-48d8-4961-a27b-e444aa24259b',
 '19588cdb-be95-4249-a7b3-729b7d8e16fc',
 'b91437a0-bc0f-4f7d-8b56-d437c16b3987',
 'a84e4e20-0e6d-4969-8a48-a208ed4909c8',
 '85c28e35-b543-4ecd-b289-7c7f0ba9928e',
 'a579da00-2c13-4452-bd99-7cd62676c7a7',
 '911de828-7cec-47c3-be82-03594f23844b',
 '00e38a5e-cc7f-4a3e-a06f-922ac65355d3',
 '95046703-8d1d-47af-952c-ab150eafe929',
 '0744e596-0015-4ed0-9849-81c1d1da6683',
 'c1bf97c4-5a5e-41f6-b38a-343c777ef1cb',
 '682da125-5823-4ef5-8207-4321af0b1c59',
 '04a19e9b-36dd-4159-a56a-34a5ea4c60c1',
 '0560356f-d247-4bb8-be46-b50905b27619',
 '9693f2de-1488-

In [141]:
# save test questions and answers
fianl_test_data = []
for data in test_data:
    fianl_test_data.extend(data)
test_questions = [q for q, a in fianl_test_data]
test_answers = [a for q, a in fianl_test_data]
with open("./submit/data/test/questions.txt", "w") as file:
    file.write("\n".join(test_questions))
with open("./submit/data/test/reference_answers.txt", "w") as file:
    file.write("\n".join(test_answers))

In [142]:
# save train questions and answers
fianl_train_data = []
for data in train_data:
    fianl_train_data.extend(data)
train_questions = [q for q, a in fianl_train_data]
train_answers = [a for q, a in fianl_train_data]
with open("./submit/data/train/questions.txt", "w") as file:
    file.write("\n".join(train_questions))
with open("./submit/data/train/reference_answers.txt", "w") as file:
    file.write("\n".join(train_answers))

# Create RAG Chain

In [143]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)

vectorstore = Chroma(embedding_function=ef, persist_directory="./pittsburgh_HF_db")

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Create retriever
vectorstore_qa = Chroma(embedding_function=ef, persist_directory="./pittsburgh_QA_1_db")
retriever_qa = vectorstore_qa.as_retriever(search_type="similarity", search_kwargs={"k": 2})

def combined_retriever(query):
    # Retrieve results from both retrievers
    results1 = retriever.get_relevant_documents(query)
    results2 = retriever_qa.get_relevant_documents(query)
    
    # Combine results (e.g., simply concatenate them)
    combined_results = results1 + results2
    
    # Optionally, you can add logic to filter or rank these combined results
    return combined_results

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    # {"context": RunnablePassthrough() | RunnableLambda(combined_retriever) | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain_2 = (
    # {"context": retriever | format_docs, "question": RunnablePassthrough()}
    {"context": RunnablePassthrough() | RunnableLambda(combined_retriever) | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain_3 = (
    # {"context": retriever | format_docs, "question": RunnablePassthrough()}
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


# Final result and tests

In [110]:
# read lines from test questions
with open('./data/Final_test_data/test_questions.csv', 'r') as file:
    lines = file.read().splitlines()


In [111]:
# check first several questions
result = [rag_chain.invoke(question) for question in lines[11:20]]
print(result)

["I don't know.", 'February 27', 'William Pitt', "I don't know.", "I don't know.", 'July 8', 'Pittsburgh Pirates', 'Andy Warhol', 'Diplodocus carnegii']


In [113]:
result = [rag_chain.invoke(question) for question in lines[11:20]]
print(result)

['Wuhan', 'February 27, 1758', 'William Pitt', "I don't know.", "I don't know.", 'July 8, 1933', 'Pittsburgh Pirates', 'Andy Warhol', 'Diplodocus carnegii.']


In [115]:
result = [rag_chain.invoke(question) for question in lines[11:20]]
print(result)

['Wuhan', 'February 27', 'William Pitt', "I don't know.", "I don't know.", 'July 8', 'Pittsburgh Pirates', 'Andy Warhol', 'Diplodocus carnegii']


In [129]:
testing_data = []
for qa_pair in test_data:
    testing_data.extend(qa_pair)

In [ ]:
test_result_no_train = [rag_chain.invoke(question) for question, answer in testing_data]

In [132]:
test_result_no_train = [rag_chain_2.invoke(question) for question, answer in testing_data]

In [144]:
test_result_no_rag = [rag_chain_3.invoke(question) for question, answer in testing_data]

In [146]:
print(len(testing_data), len(documents))
print("error rate: ",  38 / 73, 41 / 73)
for i in range(len(testing_data)):
    print(testing_data[i], test_result[i], "| ", test_result_no_train[i], "| ", test_result_no_rag[i])

73 291
error rate:  0.5205479452054794 0.5616438356164384
('When was the tradition of buggies race started?', '1920') I don't know. |  I don't know. |  I don't know.
('When did Andrew Carnegie donate $1 million for the creation of a technical institute for the city of Pittsburgh?', '1900') 1900 |  1900 |  1900
('when was the first smile :-) in an email created?', '1982') 1982 |  1982 |  1982 by Scott Fahlman
("What's the usual max speed of buggies?", '35 mph') I don't know. |  I don't know. |  I don't know.
('Which professor in CMU developed a computer algorithm that accelerates the matching of living donors?', 'Tuomas Sandholm') I don't know. |  I don't know. |  Dr. Feinberg.
('Who in CMU wrote A History of the Federal Reserve?', 'Allan Meltzer') I don't know. |  I don't know. |  Allan Meltzer
('Who started the first Girl Scout badge for negotiation?', 'Linda Babcock') I don't know. |  I don't know. |  Linda Babcock, CMU professor.
('How many Nobel Laureate has Carnegie Mellon has bee

In [116]:
result = [rag_chain.invoke(question) for question in lines[1:]]
with open("./data/Final_test_data/system_output_1.txt", "w") as file:
    file.write("\n".join(result))

In [75]:
rag_chain.invoke("Where is Pittsburgh Symphony Orchestra?")

'Heinz Hall'

In [93]:
rag_chain.invoke('What was the first school to award a degree in drama?')

'CMU'

In [91]:
rag_chain.invoke('Who was selected to take part in the 2020 Emerging Leaders Fellowship?')

"I don't know."

In [92]:
rag_chain.invoke('When was cmu founded?')

'1900'

In [94]:
rag_chain.invoke('Where is the Steelers headquartered?')

'UPMC Rooney Sports Complex'

In [117]:
rag_chain.invoke('Who serves as the Pitching Coach for the Pittsburgh Pirates?')

"I don't know."

In [120]:
rag_chain.invoke('Who is the Bench Coach for the Pittsburgh Pirates?')

"Don't know."

In [122]:
rag_chain.invoke('What is the number for Oscar Marin of the Pittsburgh Pirates?')

"I don't know."

In [125]:
context = retriever.get_relevant_documents('What is the number for Oscar Marin of the Pittsburgh Pirates?')
for document in context:
    print(document.page_content)
    print("--------")

Date: Aug 04, 2025
Time: 1:00 PM
Event Name: San Francisco GiantsatPittsburgh Pirates
Venue: PNC Park
Location: 15212, Pittsburgh, Pennsylvania, US
Price: Prices from $25
Ticket Link: https://pittsburgh.events/dates/san-francisco-giants-san-francisco-giants-at-pittsburgh-pirates-aug-04-2025-1-00-pm/
--------
Date: Aug 04, 2025
Time: 1:00 PM
Event Name: San Francisco GiantsatPittsburgh Pirates
Venue: PNC Park
Location: 15212, Pittsburgh, Pennsylvania, US
Price: Prices from $25
Ticket Link: https://pittsburgh.events/dates/san-francisco-giants-san-francisco-giants-at-pittsburgh-pirates-aug-04-2025-1-00-pm/
--------
Date: Aug 05, 2025
Time: 1:00 PM
Event Name: San Francisco GiantsatPittsburgh Pirates
Venue: PNC Park
Location: 15212, Pittsburgh, Pennsylvania, US
Price: Prices from $25
Ticket Link: https://pittsburgh.events/dates/san-francisco-giants-san-francisco-giants-at-pittsburgh-pirates-aug-05-2025-1-00-pm/
--------
Date: Aug 05, 2025
Time: 1:00 PM
Event Name: San Francisco GiantsatPit

In [19]:
rag_chain.invoke('Which school built the first green dormitory in the U.S. in 2003?')

'CMU'

In [80]:
rag_chain.invoke("What day were the Pittsburgh Steelers founded?")

'July 8, 1933'

In [109]:
rag_chain.invoke("What specific historic event does the Fort Pitt Museum in Pittsburgh focus on illustrating?")

'French and Indian War'

In [106]:
context = retriever.get_relevant_documents("Which museum in Pittsburgh specializes in materials related to bicycles and is noted as the world's largest bike museum and store?")
for document in context:
    print(document.page_content)
    print("----------")

Pittsburgh's major art museums include theAndy Warhol Museum, theCarnegie Museum of Art,The Frick Pittsburgh,Pittsburgh Center for the Arts, theMattress Factory, and theCarnegie Museum of Natural History, which has extensive dinosaur, mineral, animal, andEgyptiancollections. TheKamin Science Centerand associatedSportsWorkshas interactive technology and science exhibits. TheSenator John Heinz History Center and Western Pennsylvania Sports Museumis a Smithsonian affiliated regional history museum in the Strip District and its associatedFort Pitt Museumis in Point State Park.Soldiers and Sailors Memorial Hall and Museumin Oakland houses Western Pennsylvania military exhibits from the Civil War to present. TheChildren's Museum of Pittsburghon the North Side features interactive exhibits for children. The eclecticBayernhof Music Museumis six miles (9km) from downtown whileThe Clemente Museumis in the city's Lawrenceville section. TheCathedral of Learning'sNationality Roomsshowcase pre-19th


In [108]:
context_qa = retriever_qa.get_relevant_documents("Which museum in Pittsburgh specializes in materials related to bicycles and is noted as the world's largest bike museum and store?")
for document in context_qa:
    print(document.page_content)
    print("----------")

Q: Which famous military museum is located in Pittsburgh?
A: Soldiers and Sailors Memorial Hall and Museum
----------
Q: What's the name of the museum regraded as one of America's most colorful public art landmarks in Pittsburgh?
A: Randyland
----------
Q: What museums does Heinz History Center operate?
A: Western Pennsylvania Sports Museum;Detre Library & Archives;Meadowcroft Rockshelter and Historic Village;Fort Pitt Museum
----------


In [99]:
rag_chain.invoke('Who is Pittsburgh named after?')

'William Pitt'

In [98]:
rag_chain.invoke('What musical artist is performing at PPG Arena on October 13?')

"I don't know."

In [95]:
rag_chain.invoke('When Melia P. Tourangeau began her tenure?')

"I don't know."

In [96]:
rag_chain.invoke("When was Pittsburgh Symphony Orchestra built?")

"I don't know."

In [97]:
rag_chain.invoke("When was the first year of a quarterly Gallery Crawl?")

"I don't know."